# **Framework, Library, and API Key**

In [1]:
import pandas as pd
import json
import time
import resource 

from langchain_qdrant import Qdrant

from qdrant_client.models import (
    Distance,
    NamedSparseVector,
    NamedVector,
    SparseVector,
    PointStruct,
    SearchRequest,
    SparseIndexParams,
    SparseVectorParams,
    VectorParams,
    ScoredPoint,
)
from qdrant_client import QdrantClient

from sentence_transformers import SentenceTransformer


/workspaces/dev/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# coding=utf-8
# Copyright 2024 The GTE Team Authors and Alibaba Group.
# Licensed under the Apache License, Version 2.0 (the "License");

from collections import defaultdict
from typing import Dict, List, Tuple

import numpy as np
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer
from transformers.utils import is_torch_npu_available


class GTEEmbeddidng(torch.nn.Module):
    def __init__(self,
                 model_name: str = None,
                 normalized: bool = True,
                 use_fp16: bool = True,
                 device: str = None
                ):
        super().__init__()
        self.normalized = normalized
        if device:
            self.device = torch.device(device)
        else:
            if torch.cuda.is_available():
                self.device = torch.device("cuda")
            elif torch.backends.mps.is_available():
                self.device = torch.device("mps")
            elif is_torch_npu_available():
                self.device = torch.device("npu")
            else:
                self.device = torch.device("cpu")
                use_fp16 = False
        self.use_fp16 = use_fp16
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForTokenClassification.from_pretrained(
            model_name, trust_remote_code=True, torch_dtype=torch.float16 if self.use_fp16 else None
        )
        self.vocab_size = self.model.config.vocab_size
        self.model.to(self.device)

    def _process_token_weights(self, token_weights: np.ndarray, input_ids: list):
        # conver to dict
        result = defaultdict(int)
        unused_tokens = set([self.tokenizer.cls_token_id, self.tokenizer.eos_token_id, self.tokenizer.pad_token_id,
                             self.tokenizer.unk_token_id])
        # token_weights = np.ceil(token_weights * 100)
        for w, idx in zip(token_weights, input_ids):
            if idx not in unused_tokens and w > 0:
                token = self.tokenizer.decode([int(idx)])
                if w > result[token]:
                    result[token] = w
        return result

    @torch.no_grad()
    def encode(self,
               texts: None,
               dimension: int = None,
               max_length: int = 8192,
               batch_size: int = 16,
               return_dense: bool = True,
               return_sparse: bool = False):
        if dimension is None:
            dimension = self.model.config.hidden_size
        if isinstance(texts, str):
            texts = [texts]
        num_texts = len(texts)
        all_dense_vecs = []
        all_token_weights = []
        for n, i in enumerate(range(0, num_texts, batch_size)):
            batch = texts[i: i + batch_size]
            resulst = self._encode(batch, dimension, max_length, batch_size, return_dense, return_sparse)
            if return_dense:
                all_dense_vecs.append(resulst['dense_embeddings'])
            if return_sparse:
                all_token_weights.extend(resulst['token_weights'])
        all_dense_vecs = torch.cat(all_dense_vecs, dim=0)
        return {
            "dense_embeddings": all_dense_vecs,
            "token_weights": all_token_weights 
        }

    @torch.no_grad()
    def _encode(self,
                texts: Dict[str, torch.Tensor] = None,
                dimension: int = None,
                max_length: int = 1024,
                batch_size: int = 16,
                return_dense: bool = True,
                return_sparse: bool = False):

        text_input = self.tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=max_length)
        text_input = {k: v.to(self.model.device) for k,v in text_input.items()}
        model_out = self.model(**text_input, return_dict=True)

        output = {}
        if return_dense:
            dense_vecs = model_out.last_hidden_state[:, 0, :dimension]
            if self.normalized:
                dense_vecs = torch.nn.functional.normalize(dense_vecs, dim=-1)
            output['dense_embeddings'] = dense_vecs
        if return_sparse:
            token_weights = torch.relu(model_out.logits).squeeze(-1)
            token_weights = list(map(self._process_token_weights, token_weights.detach().cpu().numpy().tolist(),
                                                    text_input['input_ids'].cpu().numpy().tolist()))
            output['token_weights'] = token_weights

        return output

    def _compute_sparse_scores(self, embs1, embs2):
        scores = 0
        for token, weight in embs1.items():
            if token in embs2:
                scores += weight * embs2[token]
        return scores

    def compute_sparse_scores(self, embs1, embs2):
        scores = [self._compute_sparse_scores(emb1, emb2) for emb1, emb2 in zip(embs1, embs2)]
        return np.array(scores)

    def compute_dense_scores(self, embs1, embs2):
        scores = torch.sum(embs1*embs2, dim=-1).cpu().detach().numpy()
        return scores

    @torch.no_grad()
    def compute_scores(self, 
        text_pairs: List[Tuple[str, str]], 
        dimension: int = None,
        max_length: int = 1024,
        batch_size: int = 16,
        dense_weight=1.0,
        sparse_weight=0.1):
        text1_list = [text_pair[0] for text_pair in text_pairs]
        text2_list = [text_pair[1] for text_pair in text_pairs]
        embs1 = self.encode(text1_list, dimension, max_length, batch_size, return_dense=True, return_sparse=True)
        embs2 = self.encode(text2_list, dimension, max_length, batch_size, return_dense=True, return_sparse=True)
        scores = self.compute_dense_scores(embs1['dense_embeddings'], embs2['dense_embeddings']) * dense_weight + \
            self.compute_sparse_scores(embs1['token_weights'], embs2['token_weights']) * sparse_weight
        scores = scores.tolist()
        return scores


if __name__ == '__main__':
    gte = GTEEmbeddidng('Alibaba-NLP/gte-multilingual-base')
    docs =  [
        "黑龙江离俄罗斯很近",
        "哈尔滨是中国黑龙江省的省会，位于中国东北",
        "you are the hero"
    ]
    print('docs', docs)
    embs = gte.encode(docs, return_dense=True,return_sparse=True)
    print('dense vecs', embs['dense_embeddings'])
    print('sparse vecs', embs['token_weights'])


docs ['黑龙江离俄罗斯很近', '哈尔滨是中国黑龙江省的省会，位于中国东北', 'you are the hero']
dense vecs tensor([[-0.1135,  0.0662, -0.0492,  ..., -0.0178, -0.0141, -0.0204],
        [-0.1313,  0.0705, -0.0090,  ...,  0.0182, -0.0221, -0.0331],
        [ 0.0152,  0.0083, -0.0888,  ...,  0.0658,  0.0352,  0.0205]])
sparse vecs [defaultdict(<class 'int'>, {'': 1.47859525680542, '黑龙江': 3.3421802520751953, '离': 1.1636834144592285, '俄罗斯': 3.231595039367676, '很': 0.4907423257827759, '近': 1.234806776046753}), defaultdict(<class 'int'>, {'': 1.3726253509521484, '哈尔滨': 3.0335350036621094, '是中国': 0.17455829679965973, '黑龙江': 1.4853949546813965, '省': 1.5884615182876587, '会': 1.7110507488250732, ',': 0.24137377738952637, '位于': 1.0312530994415283, '中国': 0.7968422174453735, '东北': 1.5651659965515137}), defaultdict(<class 'int'>, {'you': 2.285954713821411, 'are': 1.8514695167541504, 'the': 2.6807308197021484, 'hero': 3.466928482055664})]


# **Dataset**

In [3]:
# Load FAQ json.
with open("../data/full_faq.json", "r", encoding="utf-8") as file:
    faq_data = json.load(file)

faq_data[0]


{'question': 'Di mana ada lokasi Rumah Sakit Siloam?',
 'answer': 'Ada 40 Rumah Sakit modern yang terdiri dari 13 Rumah Sakit di Jabodetabek dan 27 rumah sakit yang tersebar di Jawa, Sumatera, Kalimantan, Sulawesi, serta Bali dan Nusa Tenggara.',
 'category': 'FAQ Website'}

# **Embedding Model**

In [4]:
# Initiate Embeddings
embeddings = GTEEmbeddidng(
    "Alibaba-NLP/gte-multilingual-base"
)

url = "http://localhost:6333"
collection_name = "faq-question"
distance = Distance.COSINE
dimension = 768


In [5]:
embeddings.encode("Di mana ada lokasi Rumah Sakit Siloam?", return_dense=True, return_sparse=True)

{'dense_embeddings': tensor([[-6.0316e-02,  2.3306e-02, -1.3783e-02,  5.3060e-02, -2.7045e-03,
          -6.7353e-02,  4.6318e-02,  3.5182e-02,  3.7179e-03, -1.0241e-01,
          -4.9369e-02, -9.3286e-02,  1.9098e-02,  1.3122e-01, -9.7500e-02,
          -2.3539e-02,  7.5427e-02,  9.7216e-02, -3.0661e-02,  4.2349e-02,
           8.6679e-02,  2.6276e-02,  5.8412e-02,  6.6870e-02,  3.5499e-02,
           1.5018e-02,  6.3953e-02, -1.6361e-02, -2.4088e-02,  2.8151e-02,
          -5.8316e-02, -3.1889e-02, -1.4852e-02,  1.4932e-02,  7.6590e-02,
          -1.1079e-02,  1.2208e-03,  1.5837e-02, -1.0561e-01, -1.1516e-02,
          -2.9671e-02, -4.3666e-02,  1.4853e-02, -3.7962e-02, -1.0989e-02,
           4.6323e-03, -1.7875e-03, -6.8825e-03,  1.8846e-02,  4.8770e-02,
          -7.2868e-02, -1.6110e-02,  1.1508e-02,  8.8877e-02,  3.7868e-02,
           4.4434e-02, -9.1927e-02, -3.7677e-02,  4.2033e-02, -6.1527e-02,
           2.2259e-02, -3.3956e-03, -4.2084e-02, -3.1854e-02,  2.2892e-02,
     

In [ ]:
{'dense_embeddings': tensor([[-6.0316e-02,  2.3306e-02, -1.3783e-02,  5.3060e-02, -2.7045e-03,
           5.6918e-03,  2.3137e-02,  9.6388e-03]]),
 'token_weights': [defaultdict(int,
              {'Di': 0.30286067724227905,
               'mana': 0.2925131320953369,
               'ada': 0.6360569000244141,
               'lokasi': 1.6009650230407715,
               'Rumah': 1.2283787727355957,
               'Sakit': 2.2675766944885254,
               'Si': 1.7243058681488037,
               'lo': 2.137284755706787,
               'am': 2.3366384506225586,
               '?': 0.4936652183532715})]}

In [6]:
# Initiate client.
client = QdrantClient("localhost", port=6333)

# Show collections.
collections = client.get_collections()
print(collections)


collections=[CollectionDescription(name='faq-question')]


In [ ]:
# # Show collection details.
# collection_info = client.get_collection(collection_name="test_faq_openai")
# print(collection_info)


In [ ]:
# # Delete collection.
# client.delete_collection(collection_name="faq-question")
# client.delete_collection(collection_name="faq-question-answer")
# client.delete_collection(collection_name="faq-query-passage")


True

In [ ]:
df = 

In [102]:
def moveEmbedding(faq_data, batch_size=100):
    """
    Load FAQ data from a JSON-like list of dictionaries and upsert embeddings into Qdrant,
    processing the data in batches.
    
    Each FAQ should have the following keys: 'question', 'answer', and 'category'.
    The embedding is created from the question text.
    """
    client = QdrantClient(url=url)
    
    # Create collection if it doesn't exist.
    if not client.collection_exists(collection_name=collection_name):
        client.create_collection(
            collection_name=collection_name,
            vectors_config={
                "text-dense": VectorParams(
                    size=dimension,
                    distance=distance
                )
            },
            sparse_vectors_config={
                "text-sparse": SparseVectorParams(
                    index=SparseIndexParams(
                        on_disk=False,
                    )
                )
            },
        )
    
    total_batches = (len(faq_data) - 1) // batch_size + 1
    overall_start_time = time.time()
    
    for batch_num in range(total_batches):
        batch_start_time = time.time()
        start = batch_num * batch_size
        end = start + batch_size
        batch_faq = faq_data[start:end]
        
        texts = []
        ids = []
        payloads = []
        
        for i, faq in enumerate(batch_faq, start=start):
            # Get the question text for embedding
            text = faq['question']
            texts.append(text)
            ids.append(i)
            payloads.append({
                "page_content": text,
                "metadata": {
                    "question": faq['question'],
                    "answer": faq['answer'],
                    "category": faq['category']
                }
            })
        
        # Batch embed the texts with both dense and sparse embeddings
        batch_embedding_results = embeddings.encode(texts, return_dense=True, return_sparse=True)
        
        points = []
        for j in range(len(texts)):
            # Extract dense embedding for this text
            dense_vector = batch_embedding_results['dense_embeddings'][j].tolist()
            
            # Extract sparse embedding for this text
            sparse_indices = []
            sparse_values = []
            
            # Process the defaultdict of token weights
            token_weights = batch_embedding_results['token_weights'][j]
            for token, weight in token_weights.items():
                # Convert token to an index (hash or other method as appropriate)
                # This is a simple hash function, you might need to adjust based on your tokenizer
                token_index = abs(hash(token)) % (2**31 - 1)
                sparse_indices.append(token_index)
                sparse_values.append(weight)
                
            sparse_vector = {
                "indices": sparse_indices,
                "values": sparse_values
            }
            
            points.append(
                PointStruct(
                    id=ids[j],
                    vector={"text-dense": dense_vector, "text-sparse": sparse_vector},
                    payload=payloads[j]
                )
            )
        
        # Upsert the current batch of points into Qdrant.
        client.upsert(
            collection_name=collection_name,
            points=points
        )
        
        batch_end_time = time.time()
        batch_elapsed = batch_end_time - batch_start_time
        print(f"Processed batch {batch_num+1}/{total_batches} in {batch_elapsed:.2f} seconds")
    
    overall_end_time = time.time()
    total_elapsed = overall_end_time - overall_start_time

    # Calculate peak memory usage (in MB).
    peak_memory = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024
    print(f"Added: {len(faq_data)} FAQs in {total_elapsed:.2f} seconds, Peak Memory: {peak_memory:.2f} MB")
    

In [103]:
moveEmbedding(faq_data)

Processed batch 1/4 in 4.14 seconds
Processed batch 2/4 in 3.18 seconds
Processed batch 3/4 in 2.92 seconds
Processed batch 4/4 in 1.44 seconds
Added: 349 FAQs in 11.68 seconds, Peak Memory: 4216.48 MB


In [7]:
client = QdrantClient(
    url=url,
)
qdrant = Qdrant(
    client,
    embeddings=embeddings,
    collection_name=collection_name,
    distance_strategy=distance,
)


/tmp/ipykernel_81909/2148830083.py:4: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.1.2 and will be removed in 0.5.0. Use :class:`~QdrantVectorStore` instead.
  qdrant = Qdrant(
/workspaces/dev/venv/lib/python3.12/site-packages/langchain_qdrant/vectorstores.py:140: UserWarning: `embeddings` should be an instance of `Embeddings`.Using `embeddings` as `embedding_function` which is deprecated
  warnings.warn(


In [11]:
embeddings.encode("Di mana ada lokasi Rumah Sakit Siloam?", return_dense=True, return_sparse=True)

{'dense_embeddings': tensor([[-6.0316e-02,  2.3306e-02, -1.3783e-02,  5.3060e-02, -2.7045e-03,
          -6.7353e-02,  4.6318e-02,  3.5182e-02,  3.7179e-03, -1.0241e-01,
          -4.9369e-02, -9.3286e-02,  1.9098e-02,  1.3122e-01, -9.7500e-02,
          -2.3539e-02,  7.5427e-02,  9.7216e-02, -3.0661e-02,  4.2349e-02,
           8.6679e-02,  2.6276e-02,  5.8412e-02,  6.6870e-02,  3.5499e-02,
           1.5018e-02,  6.3953e-02, -1.6361e-02, -2.4088e-02,  2.8151e-02,
          -5.8316e-02, -3.1889e-02, -1.4852e-02,  1.4932e-02,  7.6590e-02,
          -1.1079e-02,  1.2208e-03,  1.5837e-02, -1.0561e-01, -1.1516e-02,
          -2.9671e-02, -4.3666e-02,  1.4853e-02, -3.7962e-02, -1.0989e-02,
           4.6323e-03, -1.7875e-03, -6.8825e-03,  1.8846e-02,  4.8770e-02,
          -7.2868e-02, -1.6110e-02,  1.1508e-02,  8.8877e-02,  3.7868e-02,
           4.4434e-02, -9.1927e-02, -3.7677e-02,  4.2033e-02, -6.1527e-02,
           2.2259e-02, -3.3956e-03, -4.2084e-02, -3.1854e-02,  2.2892e-02,
     

In [8]:
import numpy as np
from dataclasses import dataclass
from typing import List

# Define a simple SparseEmbedding class to match your expected type
@dataclass
class SparseEmbedding:
    indices: List[int]
    values: List[float]

def convert_embeddings(embedding_results):
    """
    Convert output from embeddings.encode() to SparseEmbedding and dense vector format
    
    Args:
        embedding_results: Output from embeddings.encode() with return_dense=True, return_sparse=True
        
    Returns:
        tuple: (query_sparse_vectors, query_dense_vector)
    """
    # Extract dense embeddings and convert to numpy array if needed
    dense_tensor = embedding_results['dense_embeddings']
    if hasattr(dense_tensor, 'numpy'):  # If it's a PyTorch tensor
        dense_vectors = [dense_tensor[i].numpy() for i in range(len(dense_tensor))]
    else:  # If it's already a numpy array
        dense_vectors = [dense_tensor[i] for i in range(len(dense_tensor))]
    
    # Extract sparse embeddings and convert to SparseEmbedding format
    sparse_vectors = []
    for token_weight_dict in embedding_results['token_weights']:
        # Sort tokens by hashed index to ensure consistent order
        indices = []
        values = []
        for token, weight in token_weight_dict.items():
            # Hash the token to get an index (same as in your embedding code)
            token_index = abs(hash(token)) % (2**31 - 1)
            indices.append(token_index)
            values.append(weight)
        
        # Create SparseEmbedding object
        sparse_vectors.append(SparseEmbedding(indices=indices, values=values))
    
    return sparse_vectors, dense_vectors

# Example usage:
def make_sparse_embedding(texts):
    """Convert texts to sparse embeddings"""
    results = embeddings.encode(texts, return_dense=True, return_sparse=True)
    sparse_vectors, _ = convert_embeddings(results)
    return sparse_vectors

def make_dense_embedding(texts):
    """Convert texts to dense embeddings"""
    results = embeddings.encode(texts, return_dense=True, return_sparse=True)
    _, dense_vectors = convert_embeddings(results)
    return dense_vectors

# Now let's update the query function to use these helpers
def queryEmbedding(query_text: str, top_k=5):
    """
    Query the Qdrant collection using both dense and sparse embeddings.
    """
    client = QdrantClient(url=url)
    
    # Get embeddings using the helper functions
    query_sparse_vectors = make_sparse_embedding([query_text])
    query_dense_vector = make_dense_embedding([query_text])
    
    # Create search requests batch
    search_results = client.search_batch(
        collection_name=collection_name,
        requests=[
            SearchRequest(
                vector=NamedVector(
                    name="text-dense",
                    vector=query_dense_vector[0].tolist(),
                ),
                limit=top_k * 2,
                with_payload=True,
            ),
            SearchRequest(
                vector=NamedSparseVector(
                    name="text-sparse",
                    vector=SparseVector(
                        indices=query_sparse_vectors[0].indices,
                        values=query_sparse_vectors[0].values,
                    ),
                ),
                limit=top_k * 2,
                with_payload=True,
            ),
        ],
    )

    return search_results


In [10]:
queryEmbedding("fasilitas di siloam")

/tmp/ipykernel_81909/4034229.py:70: DeprecationWarning: `search_batch` method is deprecated and will be removed in the future. Use `query_batch_points` instead.
  search_results = client.search_batch(


[[ScoredPoint(id=37, version=0, score=0.8191108, payload={'page_content': 'Layanan Siloam at Home apa saja yang diberikan oleh Siloam?', 'metadata': {'question': 'Layanan Siloam at Home apa saja yang diberikan oleh Siloam?', 'answer': 'Layanan Siloam at Home antara lain kunjungan perawat, kunjungan dokter umum, vaksin, perawatan luka, pemeriksaan laboratorium, dan lain-lain.', 'category': 'Siloam at Home'}}, vector=None, shard_key=None, order_value=None),
  ScoredPoint(id=38, version=0, score=0.7810262, payload={'page_content': 'Bagaimana cara melakukan pemesanan layanan Siloam at Home?', 'metadata': {'question': 'Bagaimana cara melakukan pemesanan layanan Siloam at Home?', 'answer': 'Untuk mengetahui informasi lebih lanjut atau melakukan pemesanan layanan Siloam at Home, Anda dapat mengunjungi https://www.siloamhospitals.com/siloam-at-home atau langsung menghubungi nomor WhatsApp reservasi Siloam at Home di +628111950181.', 'category': 'Siloam at Home'}}, vector=None, shard_key=None, 

In [138]:
def testing_with_query_embedding(query_text, expected, k=20):
    """
    Test the queryEmbedding function by checking if the expected result is in the top k results.
    
    Args:
        query_text (str): The query text to search for
        expected (str): The expected question text that should match
        k (int): Number of results to check
        
    Returns:
        tuple: (status, data) where status is a boolean indicating success and data contains the match or top result
    """
    # Call queryEmbedding function
    results = queryEmbedding(query_text, top_k=k)
    
    # Debug: Print the structure of results to understand its format
    if not results:
        return [False, "No result"]
    
    # Handle ScoredPoint format specifically
    if hasattr(results[0], 'payload') and 'metadata' in results[0].payload:
        for result in results:
            if result.payload['metadata']['question'] == expected:
                return [True, (result, result.score)]
        return [False, (results[0], results[0].score)]
    
    # Check the structure of results to handle it properly
    if isinstance(results, list):
        # If results is a list of dictionaries with 'question' key
        if isinstance(results[0], dict) and 'question' in results[0]:
            for result in results:
                if result['question'] == expected:
                    return [True, (result, result['score'])]
            return [False, (results[0], results[0]['score'])]
        
        # If results is a list of tuples (document, score)
        elif isinstance(results[0], tuple) and len(results[0]) == 2:
            for doc, score in results:
                if hasattr(doc, 'metadata') and doc.metadata['question'] == expected:
                    return [True, (doc, score)]
            return [False, results[0]]
    
    # If none of the above formats match, return the first result for debugging
    return [False, (results[0], getattr(results[0], 'score', 0))]

# Create test list from FAQ data
test_list = [{'query': x['question'], 'expected': x['question']} for x in faq_data]
print(f"Testing with {len(test_list)} questions")
print(f"First test case: {test_list[0]}")

# Run tests
results = []
total_tests = len(test_list)
passed_tests = 0

for t in test_list:
    try:
        [status, data] = testing_with_query_embedding(t["query"], t["expected"], k=20)
        if status:
            passed_tests += 1
        else:
            if data == "No result":
                _data = {
                    "query": t["query"],
                    "result": "No result",
                    "score": "No result"
                }
            else:
                # Handle ScoredPoint objects properly
                if hasattr(data[0], 'payload') and 'metadata' in data[0].payload:
                    result_text = data[0].payload['metadata']['question']
                    result_score = data[1]
                # Try to handle different result formats
                elif isinstance(data[0], dict) and 'question' in data[0]:
                    result_text = data[0]['question']
                    result_score = data[1]
                elif hasattr(data[0], 'metadata'):
                    result_text = data[0].metadata['question']
                    result_score = data[1]
                else:
                    result_text = str(data[0])
                    result_score = data[1]
                
                _data = {
                    "query": t["query"],
                    "result": result_text,
                    "score": result_score
                }
            results.append(_data)
    except Exception as e:
        results.append({
            "query": t["query"],
            "result": f"Error: {str(e)}",
            "score": "Error"
        })

# Display results
print(f"Tests passed: {passed_tests}/{total_tests} ({passed_tests/total_tests*100:.2f}%)")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    df = pd.DataFrame(data=results)
    display(df)

Testing with 349 questions
First test case: {'query': 'Di mana ada lokasi Rumah Sakit Siloam?', 'expected': 'Di mana ada lokasi Rumah Sakit Siloam?'}


/tmp/ipykernel_11177/4034229.py:70: DeprecationWarning: `search_batch` method is deprecated and will be removed in the future. Use `query_batch_points` instead.
  search_results = client.search_batch(


Tests passed: 0/349 (0.00%)


,query,result,score
0,Di mana ada lokasi Rumah Sakit Siloam?,"[ScoredPoint(id=0, version=0, score=0.99999994...",0
1,Apa jenis kamar rawat inap yang tersedia?,"[ScoredPoint(id=1, version=0, score=1.0000002,...",0
2,Apa standar keamanan suplai darah yang diambil...,"[ScoredPoint(id=2, version=0, score=1.0, paylo...",0
3,Fasilitas apa yang dimiliki pusat keunggulan R...,"[ScoredPoint(id=3, version=0, score=0.9999999,...",0
4,Apa standar kualitas yang dipatuhi oleh Siloam...,"[ScoredPoint(id=4, version=0, score=0.9999999,...",0
5,Metode pembayaran apa yang tersedia di Siloam ...,"[ScoredPoint(id=5, version=0, score=1.0, paylo...",0
6,Fasilitas apa yang disediakan di dalam kamar?,"[ScoredPoint(id=6, version=0, score=0.9999999,...",0
7,Apakah merokok diperbolehkan di area rumah sakit?,"[ScoredPoint(id=7, version=0, score=0.9999999,...",0
8,Berapa banyak pasien yang dirawat Siloam Hospi...,"[ScoredPoint(id=8, version=0, score=1.0, paylo...",0
9,Apakah rumah sakit Siloam menawarkan perkiraan...,"[ScoredPoint(id=9, version=0, score=1.0, paylo...",0


In [ ]:
with pd.option_context('display.max_colwidth', None):
    display(df.head())


In [16]:
new_test_list = [
    { "query": "siloam ada dimana saja", "expected": "Di mana ada lokasi Rumah Sakit Siloam?" },
    { "query": "jam buka mcu", "expected": "Jam buka MCU / Medical Check Up?" },
    { "query": "layanan kesehatan di rumah", "expected": "Apa itu Siloam at Home? Apakah Siloam memberikan layanan kesehatan di rumah atau Homecare?" },
    { "query": "cek obat beli di siloam dari aplikasi", "expected": "Halo saya mau cek obat yang saya beli dari Siloam, bisakah dari aplikasi MySiloam?" },
    { "query": "asuransi kerjasa sama dengan siloam", "expected": "Asuransi apa saja yang bekerja sama dengan Siloam?" },
    { "query": "jenis kamar rawat inap apa saja", "expected": "Apa jenis kamar rawat inap yang tersedia?" },
    { "query": "standar keamanan darah di rumah sakit seperti apa", "expected": "Apa standar keamanan suplai darah yang diambil di rumah sakit?" },
    { "query": "fasilitas pusat keunggulan rumah sakit apa saja", "expected": "Fasilitas apa yang dimiliki pusat keunggulan Rumah Sakit?" },
    { "query": "standar kualitas yang diterapkan siloam hospitals", "expected": "Apa standar kualitas yang dipatuhi oleh Siloam Hospitals?" },
    { "query": "pembayaran apa saja tersedia di siloam hospitals", "expected": "Metode pembayaran apa yang tersedia di Siloam Hospitals?" },
    { "query": "kamar rumah sakit menyediakan fasilitas apa saja", "expected": "Fasilitas apa yang disediakan di dalam kamar?" },
    { "query": "apakah merokok diperbolehkan di rumah sakit", "expected": "Apakah merokok diperbolehkan di area rumah sakit?" },
    { "query": "berapa banyak pasien di siloam hospitals tiap tahun", "expected": "Berapa banyak pasien yang dirawat Siloam Hospitals setiap tahun?" },
    { "query": "apakah siloam hospitals memberikan perkiraan biaya perawatan", "expected": "Apakah rumah sakit Siloam menawarkan perkiraan biaya perawatan?" },
    { "query": "asuransi kesehatan diterima di siloam hospitals", "expected": "Jenis asuransi kesehatan apa saja yang diterima di Siloam Hospitals?" },
    { "query": "bagaimana jika dokter pilihan tidak ada", "expected": "Bagaimana jika dokter pilihan saya tidak tersedia?" },
    { "query": "cara berkomunikasi dengan dokter bagi yang tidak fasih bahasa indonesia", "expected": "Saya tidak fasih berbahasa Indonesia. Bagaimana cara berkomunikasi dengan dokter?" },
    { "query": "barang apa yang harus dibawa ke siloam hospital", "expected": "Barang apa saja yang perlu saya bawa ke Siloam Hospital?" },
    { "query": "apakah perlu janji untuk spesialis", "expected": "Apakah saya perlu membuat janji untuk bertemu dengan spesialis?" },
    { "query": "jam kunjungan pasien di rumah sakit", "expected": "Pada jam berapa pengunjung diperbolehkan untuk mengunjungi pasien?" },
    { "query": "bolehkah kasih bunga atau hadiah ke pasien", "expected": "Apakah saya boleh memberikan bunga atau hadiah kepada pasien di Siloam Hospital?" },
    { "query": "cara hubungi pasien di ICU", "expected": "Bagaimana saya bisa menghubungi pasien di Unit Perawatan Intensif?" },
    { "query": "ada daftar hotel di sekitar siloam hospitals", "expected": "Apakah Anda memiliki daftar hotel dekat Rumah Sakit Siloam?" },
    { "query": "tamu boleh kunjungan setelah jam kerja tidak", "expected": "Apakah tamu diizinkan untuk berkunjung setelah jam kerja?" },
    { "query": "anak-anak boleh kunjungi pasien di rumah sakit", "expected": "Apakah anak-anak diperbolehkan mengunjungi pasien di Siloam Hospitals?" },
    { "query": "dokumen apa yang harus dibawa untuk ambil medical report", "expected": "Dokumen apa saja yang harus saya bawa saat ingin mengambil medical report/formulir asuransi/dokumen medis?" },
    { "query": "cara minta informasi medis", "expected": "Bagaimana cara saya meminta informasi medis saya?" },
    { "query": "cara dapat salinan rekam medis", "expected": "Bagaimana saya bisa mendapatkan salinan rekam medis saya untuk diri saya sendiri atau untuk dikirimkan kepada orang lain selain diri saya sendiri, seperti dokter, pengacara, perusahaan asuransi atau orang lain?" },
    { "query": "jam operasional mcu", "expected": "Jam buka MCU / Medical Check Up?" },
    { "query": "apakah perlu puasa sebelum mcu", "expected": "Apakah MCU / Medical Check Up perlu melakukan puasa?" },
    { "query": "berapa lama waktu untuk mcu test", "expected": "Berapa lama waktu yang dibutuhkan untuk MCU / Medical Check Up tes?" },
    { "query": "paket mcu rekomendasi apa yang terlengkap", "expected": "Apa rekomendasi paket MCU / Medical Check Up terlengkap saat ini?" },
    { "query": "apakah semua siloam hospitals sediakan mcu", "expected": "Apakah saya bisa melakukan MCU / Medical Check Up di semua Siloam?" },
    { "query": "cara daftar mcu", "expected": "Bagaimana cara mendaftar MCU / Medical Check Up?" },
    { "query": "apa yang harus dibawa untuk mcu", "expected": "Apa yang perlu dibawa ketika melakukan MCU / Medical Check Up?" },
    { "query": "bedanya daftar mcu lewat website dan telepon", "expected": "Apa bedanya mendaftar MCU / Medical Check Up lewat website dengan lewat telepon/datang langsung?" },
    { "query": "metode pembayaran untuk mcu", "expected": "Apa saja metode pembayaran pada pendaftaran MCU / Medical Check Up?" },
    { "query": "hasil mcu yang didapatkan", "expected": "Hasil apa yang saya dapatkan setelah tes MCU / Medical Check Up di Siloam?" },
    { "query": "berapa lama hasil mcu keluar", "expected": "Berapa lama hasil dari MCU / Medical Check Up tes?" },
    { "query": "persiapan sebelum datang ke mcu", "expected": "Apa saja yang harus saya lakukan sebelum datang ke MCU / Medical Check Up (persiapan yang perlu saya lakukan)?" },
    { "query": "siloam at home itu apa", "expected": "Apa itu Siloam at Home? Apakah Siloam memberikan layanan kesehatan di rumah atau Homecare?" },
    { "query": "layanan siloam at home apa saja", "expected": "Layanan Siloam at Home apa saja yang diberikan oleh Siloam?" },
    { "query": "cara pesan layanan siloam at home", "expected": "Bagaimana cara melakukan pemesanan layanan Siloam at Home?" },
    { "query": "metode pembayaran untuk siloam at home", "expected": "Apa saja metode pembayaran yang saya bisa gunakan untuk Siloam at Home?" },
    { "query": "apakah siloam at home sediakan perawat di rumah", "expected": "Apakah Siloam at Home memfasilitasi layanan perawat tinggal di rumah pasien?" },
    { "query": "cek obat dari aplikasi siloam", "expected": "Halo saya mau cek obat yang saya beli dari Siloam, bisakah dari aplikasi MySiloam?" },
    { "query": "cara pesan dokter lewat aplikasi mysiloam", "expected": "Bagaimana cara pesan dokter dari aplikasi MySiloam?" },
    { "query": "jika booking lewat aplikasi, apakah slot sesuai jadwal dokter", "expected": "Jika saya udah booking dari aplikasi MySiloam, apakah saya pasti terdaftar slot yang sesuai dengan jadwal dokter?" },
    { "query": "sudah booking di mysiloam, apakah saya terdaftar", "expected": "Saya sudah booking di aplikasi MySiloam, apakah saya sudah terdaftar?" },
    { "query": "fitur apa saja di aplikasi mysiloam", "expected": "Fitur apa sajakah yang ada di aplikasi MySiloam?" },
    { "query": "cara download aplikasi mysiloam", "expected": "Bagaimana cara download aplikasi MySiloam?" },
    { "query": "apa itu menu rekam medis di mysiloam", "expected": "Apa itu menu rekam medis pada aplikasi MySiloam?" },
    { "query": "cara aktivasi rekam medis", "expected": "Gimana caranya untuk buka atau aktivasi rekam medis?" },
    { "query": "data apa saja ada di menu rekam medis", "expected": "Data apa saja yang tersedia di menu rekam medis di aplikasi MySiloam?" },
    { "query": "data rawat inap yang ditampilkan apa saja", "expected": "Data apa saja yang tersedia untuk rawat inap?" },
    { "query": "asuransi yang kerjasama dengan siloam apa saja", "expected": "Asuransi apa saja yang bekerja sama dengan Siloam?" },
    { "query": "jika asuransi tidak kerjasama, apa yang harus dilakukan", "expected": "Apa yang harus dilakukan jika asuransi tidak bekerja sama dengan rumah sakit?" },
    { "query": "cara bayar dengan asuransi", "expected": "Bagaimana alur dalam membayar dengan asuransi?" },
    { "query": "berapa lama urusan asuransi sebelum konsultasi dokter", "expected": "Berapa lama waktu untuk pasien untuk mengurus asuransi sebelum konsultasi dengan dokter?" },
    { "query": "apa yang harus disiapkan saat daftar dengan asuransi", "expected": "Apa saja yang harus dipersiapkan saat mendaftar ke rumah sakit menggunakan asuransi?" },
    { "query": "data apa yang diperlukan saat pakai bpjs", "expected": "Data apa yang dibutuhkan saat menggunakan BPJS di Siloam Hospitals?" },
    { "query": "apakah kondisi darurat perlu surat rujukan", "expected": "Apakah kondisi gawat darurat memerlukan Surat Rujukan dari Puskesmas/Klinik seperti mendaftar rawat jalan?" },
    { "query": "rumah sakit siloam mana aja terima bpjs", "expected": "Rumah Sakit Siloam mana saja yang menerima pasien BPJS?" },
    { "query": "bisakah daftar bpjs di hari h", "expected": "Apakah saya bisa daftar BPJS di hari H?" },
    { "query": "jam operasional konsultasi bpjs", "expected": "Jam operasional untuk konsultasi BPJS?" },
    { "query": "cara reschedule appointment bpjs", "expected": "Bagaimana cara melakukan reschedule appointment apabila saya sudah melakukan pendaftaran via website?" },
    { "query": "masa berlaku surat rujukan bpjs", "expected": "Berapa lama masa berlaku Surat Rujukan yang diterbitkan Puskesmas/Klinik untuk berobat ke Rumah Sakit?" },
    { "query": "cara buat appointment bpjs", "expected": "Bagaimana cara membuat appointment/ mendaftar untuk antrian untuk BPJS?" },
    { "query": "cara dapatkan surat rujukan siloam", "expected": "Bagaimana cara mendapatkan surat rujukan untuk diperiksa oleh RS Siloam?" },
    { "query": "dimana beli produk laboratorium online", "expected": "Dimana saya dapat membeli produk laboratorium / radiologi secara online?" },
    { "query": "produk yang dicari tidak ada di website", "expected": "Produk yang saya cari tidak ada di website" },
    { "query": "cek ketersediaan prosedur di rumah sakit", "expected": "Ketersediaan prosedur pada rumah sakit" },
    { "query": "perlukah surat rujukan dokter untuk pembelian", "expected": "Apakah memerlukan surat rujukan dari dokter?" },
    { "query": "apakah jadwal di website sudah sesuai", "expected": "Apakah jadwal yang ditampilkan sudah sesuai?" },
    { "query": "biaya sudah termasuk konsultasi atau tidak", "expected": "Apakah biaya yang dibayarkan sudah termasuk dengan biaya konsultasi?" },
    { "query": "cara atur ulang jadwal", "expected": "Bagaimana jika saya ingin mengatur jadwal ulang?" },
    { "query": "ada batas atur ulang jadwal?", "expected": "Apakah ada limitasi untuk mengatur jadwal ulang?" },
    { "query": "jika butuh layanan homecare, bagaimana caranya", "expected": "Bagaimana jika saya butuh layanan homecare?" },
    { "query": "apakah bisa pesan homeservice dari semua siloam hospitals", "expected": "Apakah saya bisa melakukan pemesanan homeservice dari semua rumah sakit Siloam?" },
    { "query": "cara minta pengembalian dana", "expected": "Bagaimana jika saya ingin pengembalian dana?" },
    { "query": "pendaftaran radiologi di hari h boleh gak", "expected": "Apakah pendaftaran radiologi dapat dilakukan pada hari H?" },
    { "query": "jam operasional radiologi emergency", "expected": "Jam operasional radiologi?" },
    { "query": "jam operasional laboratorium untuk non-emergency", "expected": "Jam operasional laboratorium?" },
    { "query": "lokasi radiologi atau laboratorium di rumah sakit", "expected": "Dimanakah lokasi radiologi/laboratorium?" },
    { "query": "cara pembayaran untuk laboratorium", "expected": "Cara pembayaran?" },
    { "query": "cara pemesanan laboratorium", "expected": "Cara pemesanan?" },
    { "query": "bagaimana memasukkan diskon di pemesanan laboratorium", "expected": "Bagaimana cara memasukan diskon pada pemesanan laboratorium dan radiologi di Website?" },
    { "query": "cara pesan telekonsultasi", "expected": "Bagaimana cara melakukan pemesanan Telekonsultasi?" },
    { "query": "prosedur telekonsultasi seperti apa", "expected": "Bagaimana prosedur melakukan Telekonsultasi?" },
    { "query": "setelah pesan telekonsultasi, apa yang didapat", "expected": "Apa yang akan didapatkan setelah melakukan pemesanan Telekonsultasi?" },
    { "query": "dimana dapatkan link telekonsultasi", "expected": "Dimana saya bisa mendapatkan link Telekonsultasi?" },
    { "query": "berapa lama durasi telekonsultasi", "expected": "Berapa lama durasi Telekonsultasi?" },
    { "query": "dokter bisa apa saja di telekonsultasi", "expected": "Apa saja yang Dokter bisa lakukan dalam Teleconsultasi?" },
    { "query": "jika salah pesan telekonsultasi, bagaimana caranya", "expected": "Bagaimana jika saya salah dalam melakukan pemesanan telekonsultasi?" },
    { "query": "prosedur refund telekonsultasi", "expected": "Bagaimana prosedur untuk pengembalian dana yang sudah saya bayarkan pada pemesanan Telekonsultasi?" },
    { "query": "bisa atur ulang jadwal telekonsultasi di hari h gak", "expected": "Apakah bisa melakukan atur ulang jadwal Telekonsultasi di hari H?" },
    { "query": "jika dokter tidak hadir di telekonsultasi, apa yang dilakukan", "expected": "Bagaimana jika Dokter berhalangan untuk hadir ketika Teleconsultation saya?" },
    { "query": "setelah telekonsultasi, bagaimana alur resep obat", "expected": "Bagaimana alur jika saya di resepkan obat oleh Dokter setelah selesai melakukan Telekonsultasi?" },
    { "query": "jika ada rujukan untuk lab dan radiology, apa yang harus dilakukan", "expected": "Bagaimana jika ada rujukan untuk pemeriksaan Lab dan Radiology?" },
    { "query": "cara pesan telechat", "expected": "Bagaimana cara melakukan pemesanan Telechat?" }
]
